#                                                                                                                                                         CIA2

**Step 1: Grid Creation with Obstacles**

In [1]:
import numpy as np
import random

# Initialize the grid
grid_size = 100
grid = np.zeros((grid_size, grid_size), dtype=int)

# Randomly select start and goal positions
start = (random.randint(0, grid_size - 1), random.randint(0, grid_size - 1))
goal = (random.randint(0, grid_size - 1), random.randint(0, grid_size - 1))

# Ensure start and goal positions are not the same
while goal == start:
    goal = (random.randint(0, grid_size - 1), random.randint(0, grid_size - 1))

# Mark start and goal
grid[start] = 1
grid[goal] = 2

# Place obstacles randomly
num_obstacles = int(0.2 * grid_size * grid_size)  # 20% of grid filled with obstacles
for _ in range(num_obstacles):
    obstacle = (random.randint(0, grid_size - 1), random.randint(0, grid_size - 1))
    # Ensure obstacles don't overlap start/goal positions
    if obstacle != start and obstacle != goal:
        grid[obstacle] = -1

# Display grid (1 = start, 2 = goal, -1 = obstacle, 0 = free cell)
print("Grid with obstacles:")
print(grid)


Grid with obstacles:
[[ 0  0  0 ... -1  0  0]
 [ 0 -1  0 ...  0 -1  0]
 [ 0  0 -1 ...  0  0 -1]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0 -1 ...  0 -1  0]]


**Step 2: MDP Setup**

In [2]:
# Define actions and rewards
actions = ['up', 'down', 'left', 'right']
rewards = np.full((grid_size, grid_size), -0.1)  # Small negative reward for each step
rewards[goal] = 10  # High reward at the goal


**Step 3: Dynamic Programming (DP) Solution**

In [3]:
# Value Iteration parameters
discount_factor = 0.9
theta = 0.1  # Convergence threshold

# Initialize value function
value_function = np.zeros((grid_size, grid_size))

def get_next_state_and_reward(state, action):
    """ Get the next state and reward based on action taken from the current state. """
    x, y = state
    if action == 'up' and x > 0:
        return (x - 1, y), rewards[x - 1, y]
    elif action == 'down' and x < grid_size - 1:
        return (x + 1, y), rewards[x + 1, y]
    elif action == 'left' and y > 0:
        return (x, y - 1), rewards[x, y - 1]
    elif action == 'right' and y < grid_size - 1:
        return (x, y + 1), rewards[x, y + 1]
    return state, rewards[state]  # If action leads out of bounds

def value_iteration():
    while True:
        delta = 0
        for x in range(grid_size):
            for y in range(grid_size):
                state = (x, y)
                if grid[state] == -1:  # Skip obstacles
                    continue
                v = value_function[state]
                value_function[state] = max(
                    sum(
                        0.25 * (reward + discount_factor * value_function[next_state])
                        for next_state, reward in [get_next_state_and_reward(state, action)]
                    )
                    for action in actions
                )
                delta = max(delta, abs(v - value_function[state]))
        if delta < theta:
            break

value_iteration()
print("Value function after Value Iteration:")
print(value_function)


Value function after Value Iteration:
[[-0.030625   -0.025      -0.030625   ...  0.         -0.025
  -0.030625  ]
 [-0.025       0.         -0.025      ... -0.025       0.
  -0.025     ]
 [-0.030625   -0.025       0.         ... -0.030625   -0.025
   0.        ]
 ...
 [-0.03189063 -0.030625   -0.030625   ... -0.025      -0.030625
  -0.03189063]
 [-0.03189063 -0.030625   -0.025      ... -0.030625   -0.025
  -0.030625  ]
 [-0.030625   -0.025       0.         ... -0.025       0.
  -0.025     ]]


**Step 4: Q-Learning**

In [4]:
# Q-learning parameters
alpha = 0.1  # Learning rate
epsilon = 0.1  # Exploration rate
q_table = np.zeros((grid_size, grid_size, len(actions)))

def choose_action(state):
    """ Epsilon-greedy action selection """
    if random.uniform(0, 1) < epsilon:
        return random.choice(actions)  # Explore
    x, y = state
    return actions[np.argmax(q_table[x, y])]  # Exploit

def q_learning(episodes=1000):
    for episode in range(episodes):
        state = start
        while state != goal:
            action = choose_action(state)
            next_state, reward = get_next_state_and_reward(state, action)
            x, y = state
            nx, ny = next_state
            q_table[x, y, actions.index(action)] += alpha * (
                reward + discount_factor * np.max(q_table[nx, ny]) - q_table[x, y, actions.index(action)]
            )
            state = next_state

q_learning()
print("Q-table after Q-learning:")
print(q_table)


Q-table after Q-learning:
[[[-0.38059045 -0.38643672 -0.38254196 -0.3844813 ]
  [-0.38103618 -0.38480064 -0.38268341 -0.37983528]
  [-0.3746265  -0.38021157 -0.38328082 -0.37653833]
  ...
  [-0.47374503 -0.47680429 -0.47815157 -0.47390979]
  [-0.47786726 -0.4755546  -0.47784779 -0.47312703]
  [-0.47231657 -0.47366323 -0.47544977 -0.47353855]]

 [[-0.38469766 -0.38659519 -0.38847789 -0.38390752]
  [-0.37971916 -0.38134129 -0.38320479 -0.38371503]
  [-0.3800947  -0.37959894 -0.37780775 -0.38296753]
  ...
  [-0.47690276 -0.47158036 -0.47422398 -0.47275597]
  [-0.47673392 -0.47465617 -0.47272373 -0.47448813]
  [-0.47582881 -0.47484428 -0.47350758 -0.47399731]]

 [[-0.38471792 -0.38447064 -0.3814091  -0.38290894]
  [-0.38087809 -0.38196759 -0.38364544 -0.37852105]
  [-0.37877408 -0.38066658 -0.37799726 -0.37721205]
  ...
  [-0.47261609 -0.47342908 -0.47481544 -0.46973559]
  [-0.47605337 -0.4745259  -0.4709369  -0.47267178]
  [-0.47616959 -0.47407869 -0.4734967  -0.47359441]]

 ...

 [[-0.38

**Step 5: Benchmarking**

In [5]:
import time

def run_dp_benchmark():
    start_time = time.time()
    value_iteration()  # Run Value Iteration for DP
    dp_time = time.time() - start_time

    path_length, total_reward = get_path_and_reward(start, goal, method="dp")
    return dp_time, path_length, total_reward

def run_q_learning_benchmark():
    start_time = time.time()
    q_learning(episodes=1000)  # Run Q-learning
    q_time = time.time() - start_time

    path_length, total_reward = get_path_and_reward(start, goal, method="q_learning")
    return q_time, path_length, total_reward


In [6]:
def get_path_and_reward(start, goal, method="dp"):
    state = start
    path_length = 0
    total_reward = 0
    discount = 1.0  # To accumulate discounted rewards if needed

    while state != goal:
        x, y = state
        if method == "dp":
            # Select the action with the maximum value function
            best_action = max(
                actions, key=lambda a: get_next_state_and_reward(state, a)[1] +
                discount_factor * value_function[get_next_state_and_reward(state, a)[0]]
            )
        elif method == "q_learning":
            # Select the action with the maximum Q-value
            best_action = actions[np.argmax(q_table[x, y])]

        next_state, reward = get_next_state_and_reward(state, best_action)
        total_reward += reward * discount
        discount *= discount_factor
        state = next_state
        path_length += 1

        # Stop if stuck in a loop or can't reach goal due to obstacles
        if path_length > grid_size * grid_size:
            break

    return path_length, total_reward


In [7]:
dp_time, dp_path_length, dp_reward = run_dp_benchmark()
q_time, q_path_length, q_reward = run_q_learning_benchmark()

print("Benchmark Results:")
print(f"{'Method':<15}{'Time (s)':<10}{'Path Length':<15}{'Total Reward':<15}")
print(f"{'Dynamic Programming':<15}{dp_time:<10.4f}{dp_path_length:<15}{dp_reward:<15.2f}")
print(f"{'Q-Learning':<15}{q_time:<10.4f}{q_path_length:<15}{q_reward:<15.2f}")


Benchmark Results:
Method         Time (s)  Path Length    Total Reward   
Dynamic Programming0.3079    10001          -1.00          
Q-Learning     5.0710    31             -0.53          


**INFERENCE**

**Convergence Time:**

DP: Slower but guarantees optimal policy.

Q-Learning: Faster but depends on parameters (may need more episodes for optimal results).

**Path Length:**

DP: Shorter, as it finds the optimal path.

Q-Learning: Can be close to DP with enough training; may initially be longer.

**Total Reward:**

DP: Highest possible reward due to optimal policy.

Q-Learning: Can approximate DP’s reward with sufficient training.